# Pandas

* memory usage of Parquet read in Pandas as Arrow or NumPy arrays
  * 2021+ 11GB 6s as Arrow
  * 2021+ 39GB 16s at NumPy

In [2]:
import pandas as pd
import polars as pl
from humanfriendly import format_size, format_number
from simpler_mpl import set_commas, set_common_mpl_styles
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

display(f"Pandas {pd.__version__}, Polars {pl.__version__}")

from cell_profiler import cell_profiler as cp
%start_cell_profiler

def show_rows_memory(df, deep=False):
    """
    Show rows and memory cost of a Pandas/Polars dataframe
    `deep=True` only has an impact on Pandas numpy-backed string columns, which otherwise are undercounted
    """
    num_bytes = 0
    df_type = 'Unknown'
    try:
        num_bytes = df.estimated_size() # try Polars
        df_type = 'Polars'
    except AttributeError:
        pass
    try:
        num_bytes = df.memory_usage(deep=deep, index=False).sum() # try Pandas
        df_type = 'Pandas'
    except AttributeError:
        pass
    display(f"{df_type} df with {format_number(df.shape[0])} rows, {format_size(num_bytes)} bytes")

'Pandas 2.0.1, Polars 0.17.10'

In [2] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 203.8 MiB


In [3]:
%stop_cell_profiler

'Profiling disabled'

In [4]:
dfpda = pd.read_parquet('../test_result_2021on.parquet', dtype_backend="pyarrow")

In [5]:
show_rows_memory(dfpda)

'Pandas df with 82,012,245 rows, 11.19 GB bytes'

In [6]:
dfpda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82012245 entries, 0 to 82012244
Data columns (total 14 columns):
 #   Column             Dtype                 
---  ------             -----                 
 0   test_id            int64[pyarrow]        
 1   vehicle_id         int64[pyarrow]        
 2   test_date          timestamp[ns][pyarrow]
 3   test_class_id      int64[pyarrow]        
 4   test_type          large_string[pyarrow] 
 5   test_result        large_string[pyarrow] 
 6   test_mileage       int64[pyarrow]        
 7   postcode_area      large_string[pyarrow] 
 8   make               large_string[pyarrow] 
 9   model              large_string[pyarrow] 
 10  colour             large_string[pyarrow] 
 11  fuel_type          large_string[pyarrow] 
 12  cylinder_capacity  int64[pyarrow]        
 13  first_use_date     timestamp[ns][pyarrow]
dtypes: int64[pyarrow](5), large_string[pyarrow](7), timestamp[ns][pyarrow](2)
memory usage: 10.4 GB


In [7]:
dfpdn = pd.read_parquet('../test_result_2021on.parquet', dtype_backend="numpy_nullable")

In [8]:
show_rows_memory(dfpdn, deep=True)

'Pandas df with 82,012,245 rows, 39.66 GB bytes'

In [9]:
dfpdn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82012245 entries, 0 to 82012244
Data columns (total 14 columns):
 #   Column             Dtype         
---  ------             -----         
 0   test_id            Int64         
 1   vehicle_id         Int64         
 2   test_date          datetime64[ns]
 3   test_class_id      Int64         
 4   test_type          object        
 5   test_result        object        
 6   test_mileage       Int64         
 7   postcode_area      object        
 8   make               object        
 9   model              object        
 10  colour             object        
 11  fuel_type          object        
 12  cylinder_capacity  Int64         
 13  first_use_date     datetime64[ns]
dtypes: Int64(5), datetime64[ns](2), object(7)
memory usage: 8.9+ GB


# Speed test

In [8]:
%timeit dfpda['make'].str.len() # e.g. TOYOTA, VOLKSWAGEN

1.04 s ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [8] used -0.1 MiB RAM in 8.48s (system mean cpu 9%, single max cpu 100%), peaked 635.5 MiB above final usage, current RAM usage now 24721.8 MiB


In [9]:
%timeit dfpdn['make'].str.len()

7.59 s ± 23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [9] used 0.0 MiB RAM in 61.53s (system mean cpu 9%, single max cpu 100%), peaked 625.7 MiB above final usage, current RAM usage now 24721.8 MiB


In [10]:
%timeit dfpda['test_class_id'].max()

86.4 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
In [10] used 0.0 MiB RAM in 7.11s (system mean cpu 9%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 24721.8 MiB


In [11]:
%timeit dfpdn['test_class_id'].max()

180 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [11] used 0.4 MiB RAM in 1.58s (system mean cpu 10%, single max cpu 100%), peaked 625.6 MiB above final usage, current RAM usage now 24722.2 MiB


In [35]:
import numpy as np
np.testing.assert_approx_equal(74115.13152002622, dfpda['test_mileage'].mean())
np.testing.assert_approx_equal(74115.13152002622, dfpdn['test_mileage'].mean())

In [35] used 0.0 MiB RAM in 0.33s (system mean cpu 6%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 22834.2 MiB


In [12]:
%timeit dfpda['test_mileage'].mean()

63.1 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
In [12] used 0.0 MiB RAM in 5.25s (system mean cpu 9%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 24722.2 MiB


In [13]:
%timeit dfpdn['test_mileage'].mean()

148 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
In [13] used 0.0 MiB RAM in 12.19s (system mean cpu 8%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 24722.2 MiB


In [14]:
# IAN IS IT SANE TO TEST THE TIME OF 2 OPS - I REALLY NEED TO SEPARATE THESE...
#%timeit dfpda['test_mileage'].dropna().mean()

In [14] used 0.0 MiB RAM in 0.10s (system mean cpu 6%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 24722.2 MiB


In [15]:
#%timeit dfpdn['test_mileage'].dropna().mean()

In [15] used 0.0 MiB RAM in 0.10s (system mean cpu 12%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 24722.2 MiB


# Mileage vs first use year

### Subsample using NumPy and Arrow

**TODO**

In [11]:
#dfs = dfpdn.sample(frac=0.001)
# 
#dfs_allfuel = dfpda.query('test_result=="P"').sample(10_000)
#dfs_allfuel.shape

In [17] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25350.3 MiB


In [18]:
if False:
    #dfs = dfs_allfuel.query('fuel_type=="PE"')
    #dfs = dfs_allfuel.query('fuel_type=="DI"')
    #dfs.shape
    fig, ax = plt.subplots(constrained_layout=True, figsize=(4, 4))
    dfs.plot(kind='scatter', x='first_use_year', y='test_mileage', alpha=0.5, ax=ax)
    ax.set_xlim(xmin=1980)
    ax.set_ylim(ymax=400000)
    set_commas(ax, y_axis=True)
    set_common_mpl_styles(ax, ymin=-2000, title="MOT Year 2021+")

In [18] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25350.3 MiB


In [19]:
#dfs['first_use_year'].isna().sum()

In [19] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25350.3 MiB


In [20]:
#dfs['test_mileage'].isna().sum()

In [20] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25350.3 MiB


In [13]:
%start_cell_profiler

'Profiling enabled'

In [13] used 24283.5 MiB RAM in 222.89s (system mean cpu 8%, single max cpu 100%), peaked 11564.0 MiB above final usage, current RAM usage now 24487.3 MiB


In [15]:
df_fuel = dfpda.query('test_result=="P"').sample(10_000)
df_fuel['first_use_year'] = df_fuel['first_use_date'].dt.year
df_fuel = df_fuel[['test_mileage', 'fuel_type', 'first_use_year']]
df_fuel = df_fuel.dropna() # some of the test_mileage rows are empty
df_fuel = df_fuel[df_fuel['fuel_type'].isin(['PE', 'DI'])]

In [15] used 0.3 MiB RAM in 12.81s (system mean cpu 9%, single max cpu 100%), peaked 9620.5 MiB above final usage, current RAM usage now 24495.7 MiB


In [ ]:
#dfs_allfuel = dfpda.query('test_result=="P"').sample(10_000) 
#dfs_allfuel['first_use_year'] = dfs_allfuel['first_use_date'].dt.year
#dfs_allfuel = dfs_allfuel[['test_mileage', 'fuel_type', 'first_use_year']]
#dfs_allfuel = dfs_allfuel.dropna() # some of the test_mileage rows are empty
#df_fuel = dfs_allfuel[dfs_allfuel['fuel_type'].isin(['PE', 'DI'])]

In [ ]:
fig, ax = plt.subplots(constrained_layout=True, figsize=(4, 4))
# Note this is Pandas Arrow with Seaborn
# TODO maybe different symbols
# h/t https://stackoverflow.com/questions/56394204/pandas-groupby-scatter-plot-in-a-single-plot for categorical solution
#sns.scatterplot(x=dfs['first_use_year'], y=dfs['test_mileage'], c=pd.Categorical(dfs['fuel_type']).codes, ax=ax)
sns.scatterplot(x=df_fuel['first_use_year'], y=df_fuel['test_mileage'], hue=df_fuel['fuel_type'], hue_order=['DI', 'PE'], ax=ax)
#ax.scatter(x=dfs['first_use_year'], y=dfs['test_mileage'], c=pd.Categorical(dfs['fuel_type']).codes,)
ax.set_xlim(xmin=1960)
ax.set_ylim(ymax=400000)
set_commas(ax, y_axis=True)
set_common_mpl_styles(ax, ymin=-2000, title="MOT Year 2021+")

In [23]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9757 entries, 60722956 to 7558843
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype                
---  ------          --------------  -----                
 0   test_mileage    9757 non-null   int64[pyarrow]       
 1   fuel_type       9757 non-null   large_string[pyarrow]
 2   first_use_year  9757 non-null   int64[pyarrow]       
dtypes: int64[pyarrow](2), large_string[pyarrow](1)
memory usage: 327.5 KB
In [23] used 2.2 MiB RAM in 0.10s (system mean cpu 5%, single max cpu 30%), peaked 0.0 MiB above final usage, current RAM usage now 25352.6 MiB


# Old experiments to load in single file dumps

In [24]:
#dfpda2018 = pd.read_parquet('../test_result_2018on.parquet', dtype_backend="pyarrow")

In [24] used 0.0 MiB RAM in 0.10s (system mean cpu 8%, single max cpu 60%), peaked 0.0 MiB above final usage, current RAM usage now 25352.6 MiB


In [25]:
#dfpda2018.memory_usage(deep=True, index=False) # deep=True gives same answer as deep=False on Arrow

In [25] used 0.0 MiB RAM in 0.10s (system mean cpu 11%, single max cpu 67%), peaked 0.0 MiB above final usage, current RAM usage now 25352.7 MiB


In [26]:
#show_rows_memory(dfpda2018) # 198M rows, 27GB

In [26] used 0.0 MiB RAM in 0.10s (system mean cpu 9%, single max cpu 33%), peaked 0.0 MiB above final usage, current RAM usage now 25352.7 MiB


In [27]:
#dfpda2021 = pd.read_parquet('../test_result_2021on.parquet', dtype_backend="pyarrow")

In [27] used 0.0 MiB RAM in 0.10s (system mean cpu 13%, single max cpu 40%), peaked 0.0 MiB above final usage, current RAM usage now 25352.7 MiB


In [28]:
#show_rows_memory(dfpda2021) # 82M rows 11GB

In [28] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25352.7 MiB


In [29]:
#dfpdn2021 = pd.read_parquet('../test_result_2021on.parquet', dtype_backend="numpy_nullable")

In [29] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25352.7 MiB


In [30]:
#show_rows_memory(dfpdn2021)

In [30] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25352.7 MiB


In [31]:
#show_rows_memory(dfpdn2021, deep=True) # 82M rows 39GB

In [31] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25352.7 MiB
